In [2]:
import pickle
import pandas as pd
import numpy as np

import re

import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.preprocessing import Normalizer, StandardScaler

from tqdm import tqdm

import spacy
import requests
from scipy.spatial import distance


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Because we have a lot of strings in data, we would like to see full lenght for better examination. 
# However, this option should be switched off in the analysis.
pd.set_option('display.max_colwidth', 50)

# this one is for display all columns
pd.set_option('display.max_columns', 0)

# 1. Recipies scrapping


## 1.1 Scrapping from kuchnialidla.pl

I've chosen this portal, because there are recipies prepared by professional chiefs.

## 1.2 Import of scrapped data

Let's load the data saved in pickle. There are 3 datasets:
1. Links from kuchnialidla.pl
2. links with cousines
3. links with scrapped recipies

Unfortunately, due to website architecture, it was not possible to scrap all data in one loop.

In [3]:
FILE_ALL_LINKS = r'all_links_pickle.pkl'
FILE_ALL_CUSINE_LINKS = r'all_links_cusine_pickle.pkl'
FILE_ALL_SCRAPPED_PAGES=r'pickle_recipe_list.pkl'

with open(FILE_ALL_LINKS, mode='rb') as file:
    all_links=pickle.load(file)
    
with open(FILE_ALL_CUSINE_LINKS, mode='rb') as file:
    all_cusine_links=pickle.load(file)

with open(FILE_ALL_SCRAPPED_PAGES, mode='rb') as file:
    data=pickle.load(file)

### 1.2.1 Intro
Quick look on the dataset. How many recipies is available for analysis?

### 1.2.1. Meal type
Dataset is in dict, so let's make a dataframe.

In [4]:
df_all_links = pd.DataFrame.from_dict(all_links,orient='index')
df_all_links = df_all_links.T.melt()
df_all_links.columns=['cusine_category', 'link']
df_all_links.dropna(inplace=True)
df_all_links.drop_duplicates(subset=['link'])
df_all_links = df_all_links.rename(columns={'cusine_category':'meal_type'})

df_all_links['meal_type'].value_counts()

glowne        585
przekaski     391
ciasta        378
sniadania     326
desery        307
ciasteczka    195
salatki       183
zupy          177
fastfood      113
napoje         90
przetwory      75
Name: meal_type, dtype: int64

We see that sum of all the meal types is much higher than total number of recipies. There are duplicates that need to be examined. We will do that when we merge all the datasets into one.

### 1.2.2. Cusine style
Now, cusine style. First, let's change the labels.

In [5]:
# keys change
all_cusine_links['amerykanska'] = all_cusine_links.pop('us')
all_cusine_links['azjatycka'] = all_cusine_links.pop('azj')
all_cusine_links['czeska'] = all_cusine_links.pop('cze')
all_cusine_links['francuska'] = all_cusine_links.pop('fra')
all_cusine_links['grecka'] = all_cusine_links.pop('gre')
all_cusine_links['hiszpanska'] = all_cusine_links.pop('esp')
all_cusine_links['polska'] = all_cusine_links.pop('pol')
all_cusine_links['wloska'] = all_cusine_links.pop('ita')
all_cusine_links['brytyjska'] = all_cusine_links.pop('uk')
all_cusine_links['orientalna'] = all_cusine_links.pop('orj')
all_cusine_links['meksykanska'] = all_cusine_links.pop('mex')
all_cusine_links['tajska'] = all_cusine_links.pop('taj')

# dict to dataframe
df_all_cusine = pd.DataFrame.from_dict(all_cusine_links,orient='index')
df_all_cusine = df_all_cusine.T.melt()
df_all_cusine.columns=['cusine_style', 'link']
df_all_cusine.dropna(inplace=True)
df_all_cusine = df_all_cusine.drop_duplicates(subset=['link'])
df_all_cusine.reset_index(inplace=True, drop=True)

df_all_cusine['cusine_style'].value_counts()

polska         506
wloska         226
azjatycka      165
francuska      154
amerykanska    146
grecka          69
hiszpanska      66
orientalna      47
meksykanska     37
brytyjska       22
czeska          20
tajska           3
Name: cusine_style, dtype: int64

### 1.2.3. Recipies
Finally, lets take a look on recipies themselves. First, we need to check if characteristics that we need are available for all reicpies. 

In [6]:
# sprawdzam które przepisy nie miały wpisanych składników
missing_recipies={}
missing_ingredients = []
missing_meal_type = []
missing_instructions = []
missing_cusine = []
_trash=[]
for i in range(0,len(data)):
    try:
        _trash.append(data[i][2]['recipeIngredient'])
    except KeyError as a:
        missing_ingredients.append(data[i][2]['name'])

for i in range(0,len(data)):
    try:
        _trash.append(data[i][2]['recipeCategory'])
    except KeyError as a:
        missing_meal_type.append(data[i][2]['name'])
        
for i in range(0,len(data)):
    try:
        _trash.append(data[i][2]['recipeInstructions'])
    except KeyError as a:
        missing_instructions.append(data[i][2]['name'])
        
for i in range(0,len(data)):
    try:
        _trash.append(data[i][2]['recipeCuisine'])
    except KeyError as a:
        missing_cusine.append(data[i][2]['name'])
        
        
missing_recipies['missing_ingredients'] = missing_ingredients
missing_recipies['missing_meal_type'] = missing_meal_type
missing_recipies['missing_instructions'] = missing_instructions
missing_recipies['missing_cusine'] = missing_cusine

        
print(f'Number of missing observations for the given features:\n\n',
       f'- Ingredients: {len(missing_ingredients)}\n', 
       f'- Meal type: {len(missing_meal_type)}\n', 
       f'- Instructions: {len(missing_instructions)}\n', 
       f'- Cusine style: {len(missing_cusine)}\n')

Number of missing observations for the given features:

 - Ingredients: 7
 - Meal type: 0
 - Instructions: 0
 - Cusine style: 1534



As you see, there are a lot of missing data for cusine style. That's why we will use separate dataset (already prepared) to match missing data.

In [7]:
recipes=[]
recipe_with_errors=[]
id_number=0
for idx,recipe in enumerate(data):
    id_number+=1
    try:
        recipes.append({
#                     'id':id_number,
                    'link':data[idx][1],
                    'recipe_name':data[idx][2]['name'],
                    'meal_type_detailed':data[idx][2]['recipeCategory'],
                    'ingredients':", ".join(data[idx][2]['recipeIngredient']),
                    'instructions':data[idx][2]['recipeInstructions']
                   })

    except KeyError as e:
        recipe_with_errors.append(data[idx][2]['name'])
        
df_recipes = pd.DataFrame.from_dict(recipes)
df_recipes 

,link,recipe_name,meal_type_detailed,ingredients,instructions
0,/ravioli-z-cukinia-i-ricotta,Ravioli z cukinią i ricottą,Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz...",Przygotuj: tarkę o grubych i drobnych oczkach...
1,/galettes-bretonnes-czyli-nalesniki-po-bretonsku,"Galettes bretonnes, czyli naleśniki po bretońsku",Naleśniki na słono,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...",Przygotuj: patelnię do naleśnik&#243;w tarkę ...
2,/kluski-dyniowe-z-jogurtem-i-owocami,Kluski dyniowe z jogurtem i owocami,Kluski,"dynia piżmowa – 1 szt., purée z dyni – 500 g, ...",Przygotuj: piekarnik rozgrzany do temperatury...
3,/kurczak-w-sosie-jogurtowym,Kurczak w sosie jogurtowym,Kurczak,"filet z piersi kurczaka – 400 g, oliwa z oliwe...",Przygotuj: tarkę o drobnych oczkach ręcznik p...
4,/koreanski-dosirak-ryz-z-przystawkami-w-pudelku,Koreański dosirak – ryż z przystawkami w pudełku,Dania z ryżem,"ryż do sushi – 1 szklanka, woda (filtrowana lu...",Przygotuj: pudełko do dosirak l&#243;d w kost...
...,...,...,...,...,...
2792,/banany-smazone-w-ciescie,Banany smażone w cieście,Desery z owocami,"250 ml mleka, 120 g mąki, 1 jajko, 50 g cukru ...","Przygotowanie Mleko, mąkę jajka, cukier puder..."
2793,/serowe-placuszki-z-sosem-malinowym,Serowe placuszki z sosem malinowym,Pancakes,"300 g twarogu półtłustego (do sernika), 3 łyżk...",Przygotowanie Wszystkie składniki starannie z...
2794,/letnie-owoce-pod-kruszonka,Letnie owoce pod kruszonką,Desery z owocami,"płatki owsiane Nordwaldtaler: 0,75 szklanki, m...",Przygotowanie Płatki i ziarno słonecznika pra...
2795,/kawa-z-lodami-i-bita-smietana,Kawa z lodami i bitą śmietaną,Kawa,krem czekoladowy Mister Choc lub rozpuszczona ...,Na jedną szklankę Na dno wysokiej szklanki wk...


In our analysis we will use ingredients and instructions as a benchmark. Therefore, we need to check how the list of ingredients looks, so we can use the right method to extract ingredients names. 

Yet we see that in the instructions we have some letters and characters like "ó" marked as "&#243;" which were not recognized correctly. But since instructions are not key data for analysis, I will leave it as it is for now. We will work on that if necessary later.

Now, let's focus on ingredients. How they look? 

In [8]:
df_recipes['ingredients'].sample(n=1)

909    1 bakłażan, 1 opakowanie kindziuka z pieprzem,...
Name: ingredients, dtype: object

What all of them have in common is comma ingredient separation. Unfortunately, as you see, apart from that, they are written with no clear pattern. There are numbers, special characters, additional words, measurement units and polish forms. 

In great majority measurements are at the end of the phrase, separated by "-":

In [9]:
df_recipes['ingredients'].iloc[2406]

'mąka pszenna – 175 g, orzechy laskowe – 75 g, proszek do pieczenia – 1 łyżeczka, masło – 125 g, cukier – 125 g, sól, jaja – 2 szt., mleko – 50 ml, czekolada biała – 200 g, czekolada deserowa – 30 g'

But in some recipies, they are at front with no special character separation.

In [10]:
df_recipes['ingredients'].iloc[2173]

'5 jajek, umytych, rozmiar L, białka oddzielone od żółtek, 120g cukru, 110 g mąki pszennej, 20 g mączki ziemniaczanej, 1 płaska łyżeczka proszku do pieczenia, 1/2 łyżeczki sody, 50 g kakao, 40 ml oleju, 3 łyżki wody, 60 g cukru, 60 ml wody, 1 opakowanie cukru wanilinowego, 50 ml wódki (opcjonalnie), 120g cukru, 80 ml wody, gorącej, 200 g orzeszków arachidowych solonych, 400 g masła, miękkiego, 110 g cukru pudru, 454 g masła orzechowego (z orzechów ziemnych o temperaturze pokojowej)'

Since we have over 2000 recipies, we are not able to examin all of them to find any easy rule to extract only product names that we need. Therefore, we will start simple. 

In [11]:
instructions=[]
recipe_names = []
dataset=[]
meal_type=[]
links=[]
ingredients_raw = []
ingredients_initial_clean=[]
meal_type_detailed= []
ingredients_count = []

for idx,recipe_list in enumerate(recipes):
    recipe_names.append(recipes[idx]['recipe_name'])
    links.append(recipes[idx]['link'])    
    ingredients_raw.append(recipes[idx]['ingredients'])
    instructions.append(recipes[idx]['instructions'])
    meal_type_detailed.append(recipes[idx]['meal_type_detailed'])
    
#     first we need to split each ingredient group by comma
    ingredients_regex_cleaned = recipes[idx]['ingredients']
    ingredients_regex_cleaned = ingredients_regex_cleaned.split(", ")
    
# return how many ingredients are needed for recipe
    ingredients_count.append(len(ingredients_regex_cleaned))

#     then, take only first part if there is "-" in the group
    ingredients_regex_cleaned = [re.split('–|:',i)[0].strip(' ') for i in ingredients_regex_cleaned]
    ingredients_regex_cleaned = str(", ".join(ingredients_regex_cleaned))

#     now we get rid of the numbers if they are at the beginning of the ingredient group
    ingredients_regex_cleaned = re.sub(r"(\d+[,-])?(\d*)|%", "", ingredients_regex_cleaned)

    ingredients_initial_clean.append(ingredients_regex_cleaned)

    
data_dict={}
data_dict["link"]=links
# data_dict["recipe_name"]=recipe_names
data_dict["raw_ingredients"]=ingredients_raw
data_dict["cleaned_ingredients"]=ingredients_initial_clean
data_dict["ingredients_count"]=ingredients_count
# data_dict["meal_type_detailed"]=meal_type_detailed
# data_dict["instructions"]=instructions


# data_dict["ingredients_regex"]=X
df_data_dict = pd.DataFrame(data_dict)
df_data_dict

,link,raw_ingredients,cleaned_ingredients,ingredients_count
0,/ravioli-z-cukinia-i-ricotta,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz...","cukinia, oliwa z oliwek, czosnek, świeża bazyl...",22
1,/galettes-bretonnes-czyli-nalesniki-po-bretonsku,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...","mąka pszenna, mąka gryczana, mleko, jajka, wod...",12
2,/kluski-dyniowe-z-jogurtem-i-owocami,"dynia piżmowa – 1 szt., purée z dyni – 500 g, ...","dynia piżmowa, purée z dyni, mąka pszenna, mąk...",17
3,/kurczak-w-sosie-jogurtowym,"filet z piersi kurczaka – 400 g, oliwa z oliwe...","filet z piersi kurczaka, oliwa z oliwek, masło...",16
4,/koreanski-dosirak-ryz-z-przystawkami-w-pudelku,"ryż do sushi – 1 szklanka, woda (filtrowana lu...","ryż do sushi, woda (filtrowana lub niskozminer...",24
...,...,...,...,...
2792,/banany-smazone-w-ciescie,"250 ml mleka, 120 g mąki, 1 jajko, 50 g cukru ...","ml mleka, g mąki, jajko, g cukru pudru, g...",7
2793,/serowe-placuszki-z-sosem-malinowym,"300 g twarogu półtłustego (do sernika), 3 łyżk...","g twarogu półtłustego (do sernika), łyżki mą...",11
2794,/letnie-owoce-pod-kruszonka,"płatki owsiane Nordwaldtaler: 0,75 szklanki, m...","płatki owsiane Nordwaldtaler, masło Pilos do k...",12
2795,/kawa-z-lodami-i-bita-smietana,krem czekoladowy Mister Choc lub rozpuszczona ...,krem czekoladowy Mister Choc lub rozpuszczona ...,4


## 1.3. Dataset preparation

### 1.3.1. Datasets merge

In [12]:
# łączę bazę ze zescrapowanymi przepisami z bazą w której mam informację o rodzaju posiłku
df_working = pd.merge(df_all_links, df_recipes, on='link')

df_working = pd.merge(df_working, df_data_dict, on='link', how='inner')

# nową bazę łączę z bazą, w której mam styl kuchni
df_working = pd.merge(df_working, df_all_cusine, on='link', how='left')



# usuwam duplikaty, które się pojawiły przy scrapowaniu
df_working = df_working.drop_duplicates()

# resetuję indeksy
df_working.reset_index(drop=True, inplace=True)
df_working

,meal_type,link,recipe_name,meal_type_detailed,ingredients,instructions,raw_ingredients,cleaned_ingredients,ingredients_count,cusine_style
0,glowne,/ravioli-z-cukinia-i-ricotta,Ravioli z cukinią i ricottą,Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz...",Przygotuj: tarkę o grubych i drobnych oczkach...,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz...","cukinia, oliwa z oliwek, czosnek, świeża bazyl...",22,wloska
1,glowne,/galettes-bretonnes-czyli-nalesniki-po-bretonsku,"Galettes bretonnes, czyli naleśniki po bretońsku",Naleśniki na słono,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...",Przygotuj: patelnię do naleśnik&#243;w tarkę ...,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...","mąka pszenna, mąka gryczana, mleko, jajka, wod...",12,francuska
2,sniadania,/galettes-bretonnes-czyli-nalesniki-po-bretonsku,"Galettes bretonnes, czyli naleśniki po bretońsku",Naleśniki na słono,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...",Przygotuj: patelnię do naleśnik&#243;w tarkę ...,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...","mąka pszenna, mąka gryczana, mleko, jajka, wod...",12,francuska
3,glowne,/kluski-dyniowe-z-jogurtem-i-owocami,Kluski dyniowe z jogurtem i owocami,Kluski,"dynia piżmowa – 1 szt., purée z dyni – 500 g, ...",Przygotuj: piekarnik rozgrzany do temperatury...,"dynia piżmowa – 1 szt., purée z dyni – 500 g, ...","dynia piżmowa, purée z dyni, mąka pszenna, mąk...",17,polska
4,glowne,/kurczak-w-sosie-jogurtowym,Kurczak w sosie jogurtowym,Kurczak,"filet z piersi kurczaka – 400 g, oliwa z oliwe...",Przygotuj: tarkę o drobnych oczkach ręcznik p...,"filet z piersi kurczaka – 400 g, oliwa z oliwe...","filet z piersi kurczaka, oliwa z oliwek, masło...",16,NaN
...,...,...,...,...,...,...,...,...,...,...
2785,desery,/mus-czekoladowy-z-malinami,Mus czekoladowy z malinami,Desery czekoladowe,"100 g świeżych lub mrożonych malin, 15 g cukru...",Przygotowanie Żelatynę moczymy w zimnej wodzi...,"100 g świeżych lub mrożonych malin, 15 g cukru...","g świeżych lub mrożonych malin, g cukru pudr...",8,NaN
2786,desery,/jablka-zapiekane-z-zurawina,Jabłka zapiekane z żurawiną,Desery z owocami,"50 g białek jaj (2 białka jaj), 70 g cukru, 30...",Masa bezowa Białka ubijamy na sztywną masę. D...,"50 g białek jaj (2 białka jaj), 70 g cukru, 30...","g białek jaj ( białka jaj), g cukru, g migd...",5,NaN
2787,desery,/panna-cotta-waniliowa-z-owocami-lasu,Panna cotta waniliowa z owocami lasu,Desery z owocami,"250 ml śmietanki 30%, 250 ml mleka 3,2%, 80-10...","Przygotowanie Mleko, śmietankę i cukier podgr...","250 ml śmietanki 30%, 250 ml mleka 3,2%, 80-10...","ml śmietanki , ml mleka , g cukru kryształu...",11,wloska
2788,desery,/letnie-owoce-pod-kruszonka,Letnie owoce pod kruszonką,Desery z owocami,"płatki owsiane Nordwaldtaler: 0,75 szklanki, m...",Przygotowanie Płatki i ziarno słonecznika pra...,"płatki owsiane Nordwaldtaler: 0,75 szklanki, m...","płatki owsiane Nordwaldtaler, masło Pilos do k...",12,NaN


This is our working database, let's save it for later use. 

In [13]:
df_working.to_pickle("df_working_pickle.pkl")

df_working = pd.read_pickle("df_working_pickle.pkl")
df_working

,meal_type,link,recipe_name,meal_type_detailed,ingredients,instructions,raw_ingredients,cleaned_ingredients,ingredients_count,cusine_style
0,glowne,/ravioli-z-cukinia-i-ricotta,Ravioli z cukinią i ricottą,Bez mięsa,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz...",Przygotuj: tarkę o grubych i drobnych oczkach...,"cukinia – 1 szt., oliwa z oliwek – 1 łyżka, cz...","cukinia, oliwa z oliwek, czosnek, świeża bazyl...",22,wloska
1,glowne,/galettes-bretonnes-czyli-nalesniki-po-bretonsku,"Galettes bretonnes, czyli naleśniki po bretońsku",Naleśniki na słono,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...",Przygotuj: patelnię do naleśnik&#243;w tarkę ...,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...","mąka pszenna, mąka gryczana, mleko, jajka, wod...",12,francuska
2,sniadania,/galettes-bretonnes-czyli-nalesniki-po-bretonsku,"Galettes bretonnes, czyli naleśniki po bretońsku",Naleśniki na słono,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...",Przygotuj: patelnię do naleśnik&#243;w tarkę ...,"mąka pszenna – 200 g, mąka gryczana – 100 g, m...","mąka pszenna, mąka gryczana, mleko, jajka, wod...",12,francuska
3,glowne,/kluski-dyniowe-z-jogurtem-i-owocami,Kluski dyniowe z jogurtem i owocami,Kluski,"dynia piżmowa – 1 szt., purée z dyni – 500 g, ...",Przygotuj: piekarnik rozgrzany do temperatury...,"dynia piżmowa – 1 szt., purée z dyni – 500 g, ...","dynia piżmowa, purée z dyni, mąka pszenna, mąk...",17,polska
4,glowne,/kurczak-w-sosie-jogurtowym,Kurczak w sosie jogurtowym,Kurczak,"filet z piersi kurczaka – 400 g, oliwa z oliwe...",Przygotuj: tarkę o drobnych oczkach ręcznik p...,"filet z piersi kurczaka – 400 g, oliwa z oliwe...","filet z piersi kurczaka, oliwa z oliwek, masło...",16,NaN
...,...,...,...,...,...,...,...,...,...,...
2785,desery,/mus-czekoladowy-z-malinami,Mus czekoladowy z malinami,Desery czekoladowe,"100 g świeżych lub mrożonych malin, 15 g cukru...",Przygotowanie Żelatynę moczymy w zimnej wodzi...,"100 g świeżych lub mrożonych malin, 15 g cukru...","g świeżych lub mrożonych malin, g cukru pudr...",8,NaN
2786,desery,/jablka-zapiekane-z-zurawina,Jabłka zapiekane z żurawiną,Desery z owocami,"50 g białek jaj (2 białka jaj), 70 g cukru, 30...",Masa bezowa Białka ubijamy na sztywną masę. D...,"50 g białek jaj (2 białka jaj), 70 g cukru, 30...","g białek jaj ( białka jaj), g cukru, g migd...",5,NaN
2787,desery,/panna-cotta-waniliowa-z-owocami-lasu,Panna cotta waniliowa z owocami lasu,Desery z owocami,"250 ml śmietanki 30%, 250 ml mleka 3,2%, 80-10...","Przygotowanie Mleko, śmietankę i cukier podgr...","250 ml śmietanki 30%, 250 ml mleka 3,2%, 80-10...","ml śmietanki , ml mleka , g cukru kryształu...",11,wloska
2788,desery,/letnie-owoce-pod-kruszonka,Letnie owoce pod kruszonką,Desery z owocami,"płatki owsiane Nordwaldtaler: 0,75 szklanki, m...",Przygotowanie Płatki i ziarno słonecznika pra...,"płatki owsiane Nordwaldtaler: 0,75 szklanki, m...","płatki owsiane Nordwaldtaler, masło Pilos do k...",12,NaN


In [ ]:
df_working['ingredients_count'].hist(bins=50)

### 1.3.2 Final cleaning

#### 1.3.2.1 Recipies are assigned to more than 1 meal type

W trakcie przeglądania bazy, da się zaobserwowć, że niektóre przepisy są przypasowane do więcej niż jednej kategorii. Na przykładzie poniżej widać, że Chutney jabłkowy jest zaklasyfikowany do przetworów i przekąsek

In [14]:
df_working[df_working['link']=='/3-sposoby-na-chutney-jablkowy']

,meal_type,link,recipe_name,meal_type_detailed,ingredients,instructions,raw_ingredients,cleaned_ingredients,ingredients_count,cusine_style
1248,przetwory,/3-sposoby-na-chutney-jablkowy,3 sposoby na chutney jabłkowy,Przetwory owocowe,"jabłka – 2 szt., cukier – 200 g, biały ocet wi...",Przygotuj słoiczki Chutney z kolendrą W garnk...,"jabłka – 2 szt., cukier – 200 g, biały ocet wi...","jabłka, cukier, biały ocet winny, kolendra w z...",19,azjatycka
1249,przekaski,/3-sposoby-na-chutney-jablkowy,3 sposoby na chutney jabłkowy,Przetwory owocowe,"jabłka – 2 szt., cukier – 200 g, biały ocet wi...",Przygotuj słoiczki Chutney z kolendrą W garnk...,"jabłka – 2 szt., cukier – 200 g, biały ocet wi...","jabłka, cukier, biały ocet winny, kolendra w z...",19,azjatycka


In [17]:
pd.concat(i for _, i in df_working.groupby("recipe_name") if len(i) > 1)

,meal_type,link,recipe_name,meal_type_detailed,ingredients,instructions,raw_ingredients,cleaned_ingredients,ingredients_count,cusine_style
2468,ciasteczka,/2-sposoby-na-batony-po-treningu,2 sposoby na batony po treningu,Ciasteczka owsiane,"płatki owsiane – 1,5 szklanki, kasza jaglana e...",Przygotuj 2 blachy o wymiarach 15 x 15 cm mik...,"płatki owsiane – 1,5 szklanki, kasza jaglana e...","płatki owsiane, kasza jaglana ekspandowana, ma...",16,NaN
2469,desery,/2-sposoby-na-batony-po-treningu,2 sposoby na batony po treningu,Ciasteczka owsiane,"płatki owsiane – 1,5 szklanki, kasza jaglana e...",Przygotuj 2 blachy o wymiarach 15 x 15 cm mik...,"płatki owsiane – 1,5 szklanki, kasza jaglana e...","płatki owsiane, kasza jaglana ekspandowana, ma...",16,NaN
623,glowne,/2-sposoby-na-pizze-z-brokulu-i-kalafiora,2 sposoby na pizzę - z brokułu i kalafiora,Dania z warzyw,"kalafior – 0,5 główki, czosnek – 1 ząbek, ser ...",Pizza na spodzie z kalafiora Przygotuj piekar...,"kalafior – 0,5 główki, czosnek – 1 ząbek, ser ...","kalafior, czosnek, ser mozzarella light, jajko...",32,NaN
624,fastfood,/2-sposoby-na-pizze-z-brokulu-i-kalafiora,2 sposoby na pizzę - z brokułu i kalafiora,Dania z warzyw,"kalafior – 0,5 główki, czosnek – 1 ząbek, ser ...",Pizza na spodzie z kalafiora Przygotuj piekar...,"kalafior – 0,5 główki, czosnek – 1 ząbek, ser ...","kalafior, czosnek, ser mozzarella light, jajko...",32,NaN
625,glowne,/3-dania-fit-sniadanie-obiad-deser,"3 dania fit! Śniadanie, obiad, deser",Makaron z pesto,"chleb razowy – 2 kromki, awokado – 1 szt., pom...",Tosty z awokado Krok 1: Podpiekamy kromki chl...,"chleb razowy – 2 kromki, awokado – 1 szt., pom...","chleb razowy, awokado, pomidor, jaja, parmezan...",34,NaN
...,...,...,...,...,...,...,...,...,...,...
457,fastfood,/sniadaniowe-burrito,Śniadaniowe burrito,Dania mięsne,"śmietana 18% – 150 g, pieprz cayenne, limonka ...",Krok 1: Przygotowujemy sos Śmietanę mieszamy ...,"śmietana 18% – 150 g, pieprz cayenne, limonka ...","śmietana , pieprz cayenne, limonka, miód, kieł...",11,meksykanska
1392,sniadania,/sniadaniowy-chlebek-marchewkowy,Śniadaniowy chlebek marchewkowy,Pieczywo,"marchewka – 2 szt., olej – 2 łyżki, suszone dr...","PRZYGOTUJ tarkę o grubych oczkach, robot kuch...","marchewka – 2 szt., olej – 2 łyżki, suszone dr...","marchewka, olej, suszone drożdże instant, ciep...",13,NaN
1393,przekaski,/sniadaniowy-chlebek-marchewkowy,Śniadaniowy chlebek marchewkowy,Pieczywo,"marchewka – 2 szt., olej – 2 łyżki, suszone dr...","PRZYGOTUJ tarkę o grubych oczkach, robot kuch...","marchewka – 2 szt., olej – 2 łyżki, suszone dr...","marchewka, olej, suszone drożdże instant, ciep...",13,NaN
1741,sniadania,/swiateczny-pasztet-z-pistacjami-i-zurawina,Świąteczny pasztet z pistacjami i żurawiną,Śniadania,"80 dag mięsa z udek kurczaka, 20 dag piersi ku...","Przygotowanie Mięso drobiowe myjemy, zalewamy...","80 dag mięsa z udek kurczaka, 20 dag piersi ku...","dag mięsa z udek kurczaka, dag piersi kurcza...",19,polska


Grupuję linki, żeby sprawdzić, które przepisy mają więcej niż jedną kategorię posiłku

In [18]:
duplicated_categories = df_working.groupby(by = ['link']).count()
duplicated_categories = duplicated_categories[duplicated_categories['recipe_name']>1]

print(f'{len(duplicated_categories)} przepisów przynależy do więcej niż jednego typu posiłku')

455 przepisów przynależy do więcej niż jednego typu posiłku


In [20]:
df_working.isnull().sum()

meal_type                 0
link                      0
recipe_name               0
meal_type_detailed        0
ingredients               0
instructions              0
raw_ingredients           0
cleaned_ingredients       0
ingredients_count         0
cusine_style           1529
dtype: int64